In [14]:
supp = np.copy(total_rewards)

In [10]:
total_rewards = np.copy(supp)

In [15]:
from core.DQNAgent import DQNAgent
from core.CartPoleNeuralNetwork import CartPoleNeuralNetwork
import gym
import numpy as np
import torch
from tqdm import tqdm
import glob
import os
from IPython.display import Video
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from itertools import cycle
from CartPoleWrapper import CartPoleWrapper
import sys


import pyvirtualdisplay
_display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
_ = _display.start()

def plot_videos(videos_path='.', output_file_path='.'):
  stringa = 'ffmpeg -i \"concat:'
  elenco_video = glob.glob(f'{videos_path}/*.mp4')
  if len(elenco_video) == 0:
      print('0 mp4 found in this path')
      return
  elenco_file_temp = []
  for f in elenco_video:
    file = videos_path + '/temp' + str(elenco_video.index(f) + 1) + '.ts'
    os.system('ffmpeg -i ' + f + ' -c copy -bsf:v h264_mp4toannexb -f mpegts ' + file)
    elenco_file_temp.append(file)
  for f in elenco_file_temp:
    stringa += f
    if elenco_file_temp.index(f) != len(elenco_file_temp)-1:
      stringa += '|'
    else:
      stringa += f'\" -c copy  -bsf:a aac_adtstoasc {output_file_path}'
  os.system(stringa)
  display(Video(output_file_path, embed=True))

def plot_metrics():
    cycol = cycle('bgrcmk')
    f, (ax1, ax2, ax3) = plt.subplots(1, 3)

    samples = 20
    total_rewards.append([np.array(total_rewards[-(total_rewards%samples):]).mean()] * (total_rewards/samples - total_rewards%samples))
    total_rewards = np.array(total_rewards).reshape(20, -1).mean(axis=1)

    episodes = []
    for i in range(n_episodes):
        if i % (n_episodes / len(total_rewards)) == 0:
            episodes.append(i)
    
    
    ax1.set_xlabel('episodes')
    ax1.set_ylabel('total_rewards')
    ax1.plot(episodes, total_rewards, c=next(cycol))
    '''
    ax2.set_xlabel('episodes')
    ax2.set_ylabel('number_steps')
    ax2.plot(episodes, number_steps, c=next(cycol))

    ax3.set_xlabel('episodes')
    ax3.set_ylabel('cost_function_means')
    ax3.plot(episodes, cost_means, c=next(cycol))
    '''
    f.tight_layout()
plot_metrics()

UnboundLocalError: local variable 'total_rewards' referenced before assignment

In [12]:
np.random.seed(0)
agent = DQNAgent(env=CartPoleWrapper(gym.make("CartPole-v1")), nn=CartPoleNeuralNetwork())

DISCOUNT_FACTOR = 0.95
LEARNING_RATE = 0.0001

total_rewards = []
number_steps = []
cost_means = []
n_episodes = 0

In [13]:
while n_episodes < 10000:
    total_reward, steps, mean_cost_function = agent.start_episode_and_evaluate(DISCOUNT_FACTOR, LEARNING_RATE, 0, render=False, optimize=False)
    print(f'\nn_episodes_training: {n_episodes}\tsteps: {steps}\ttotal_reward: {total_reward}\tmean_cost_function:{mean_cost_function}', flush = True)
    total_rewards.append(total_reward)
    number_steps.append(steps)
    cost_means.append(mean_cost_function)

    for i in tqdm(range(10), 'learning...'):
        agent.start_episode(DISCOUNT_FACTOR, LEARNING_RATE, 1)
    n_episodes += i+1

    if n_episodes % 110 == 0:
        agent.save(f'saves/data{n_episodes}.nn')



learning...: 100%|██████████| 10/10 [00:00<00:00, 154.26it/s]
n_episodes_training: 11	steps: 13	total_reward: 11.0	mean_cost_function:0.6580767237800025

learning...: 100%|██████████| 10/10 [00:00<00:00, 161.41it/s]
n_episodes_training: 22	steps: 10	total_reward: 8.0	mean_cost_function:0.5284366969845323

learning...: 100%|██████████| 10/10 [00:00<00:00, 260.28it/s]
n_episodes_training: 33	steps: 9	total_reward: 7.0	mean_cost_function:0.535577931353897

learning...: 100%|██████████| 10/10 [00:00<00:00, 182.77it/s]
n_episodes_training: 44	steps: 10	total_reward: 8.0	mean_cost_function:0.6381844891608434

learning...: 100%|██████████| 10/10 [00:00<00:00, 242.86it/s]
n_episodes_training: 55	steps: 9	total_reward: 7.0	mean_cost_function:0.5386842904109761

learning...: 100%|██████████| 10/10 [00:00<00:00, 165.18it/s]
n_episodes_training: 66	steps: 10	total_reward: 8.0	mean_cost_function:0.5330615548956976

learning...: 100%|██████████| 10/10 [00:00<00:00, 118.40it/s]
n_episodes_training: 7

KeyboardInterrupt: 

In [ ]:

agent.env = gym.wrappers.Monitor(agent.env, 'recording/tmp-videos', force=True, video_callable=lambda episode_id: True)
agent.nn.load('saves/buona.nn')

for i in range(20):
    total_reward, steps, mean_cost_function = agent.start_episode_and_evaluate(DISCOUNT_FACTOR, LEARNING_RATE, 0, render=True, optimize=False)
    print(f'{i}\t{steps}\t{total_reward}\t{mean_cost_function}')
env.close()

agent.env = agent.env.env
plot_videos('recording/tmp-videos', 'recording/0-episodes.mp4')

evaluation

train 1 000 (with %10 evaluation)

evaluation

train 9 000 - 10 000 (with %90 evaluation)

evaluation

train 90 000 - 100 000

evaluation

train 900 000 - 1 000 0000

evaluation